In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from scipy import stats
import random
import warnings
warnings.filterwarnings("ignore")
get_ipython().magic('matplotlib inline')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [16]:
# Load in the data and drop extraneous columns
df = pd.read_csv('NeuronDataMaster2.csv')
df=df.dropna(axis=1, how='all')


In [45]:
#Defining a sort function to be used later
def Sort(DataType, df):
    df=df.sort_values(DataType)
    return df

In [4]:
#Making dataframes to hold the principal cells and the interneurons 
Principal_Cells = df.where(df.Primary_Cell_Class == 'principal cell').dropna()
Interneuron_Cells = df.where(df.Primary_Cell_Class == 'interneuron').dropna()

In [5]:
print('Principal Cells average Number of Branches: ', Principal_Cells.Number_Branches.mean())
print('Interneuron Cells average Number of Branches: ', Interneuron_Cells.Number_Branches.mean())
print('Principal Cells Number of Branches Standard Deviation: ', Principal_Cells.Number_Branches.std())
print('Interneuron Cells Number of Branches Standard Deviation: ', Interneuron_Cells.Number_Branches.std())
print(' ')
print('Principal Cells average Volume: ', Principal_Cells.Total_Volume.mean())
print('Interneuron Cells average Volume: ', Interneuron_Cells.Total_Volume.mean())
print('Principal Cells Volume Standard Deviation: ', Principal_Cells.Total_Volume.std())
print('Interneuron Cells Volume Standard Deviation: ', Interneuron_Cells.Total_Volume.std())
print(' ')
print('Principal Cells average Surface: ', Principal_Cells.Total_Surface.mean())
print('Interneuron Cells average Surface: ', Interneuron_Cells.Total_Surface.mean())
print('Principal Cells Surface Standard Deviation: ', Principal_Cells.Total_Surface.std())
print('Interneuron Cells Surface Standard Deviation: ', Interneuron_Cells.Total_Surface.std())

Principal Cells average Number of Branches:  80.44283876825301
Interneuron Cells average Number of Branches:  132.38746223564954
Principal Cells Number of Branches Standard Deviation:  137.56756073653898
Interneuron Cells Number of Branches Standard Deviation:  285.2310406313767
 
Principal Cells average Volume:  8016.71154800638
Interneuron Cells average Volume:  4662.187382930531
Principal Cells Volume Standard Deviation:  38786.131910447446
Interneuron Cells Volume Standard Deviation:  22039.75496022572
 
Principal Cells average Surface:  13176.931130362425
Interneuron Cells average Surface:  9103.962392371606
Principal Cells Surface Standard Deviation:  35460.00283942518
Interneuron Cells Surface Standard Deviation:  22634.75087141071


In [88]:
#In order to compare the mean value of two populations, we must use a hypothesis test. In this test, we use the difference of the 
#two calculated means and an adjusted standard deviation to see if 0 is within the interval of 90% confidence. If 0 is contained
#in the interval, then the test will return false, indicating that no conclusion can be reached (i.e. we cannot reject our null
#hypothesis that the are the same).Otherwise, the test indicates there is a significant difference between the two populations
def Hypothesis_Test(data1, data2 ):
    #All of our Hypothesis tests will use a signifigance level of 0.05 or 90% confidence. 
    size1 = len(data1.index)
    size2 = len(data2.index)
    Delta = data1.mean()-data2.mean()
    Std = math.pow(math.pow(data1.std(),2)/size1+math.pow(data2.std(),2)/size2,.5)
    if (((Delta - 2.132*Std>0)and(Delta +2.132*Std>0))or((Delta - 2.132*Std<0)and(Delta +2.132*Std<0))):
        return True
    else:
        return False

In [7]:
print('Significant Difference in Number of Branches: ', Hypothesis_Test(Principal_Cells.Number_Branches, Interneuron_Cells.Number_Branches))
print('Significant Difference in Volume: ',Hypothesis_Test(Principal_Cells.Total_Volume, Interneuron_Cells.Total_Volume) )
print('Significant Difference in Surface: ', Hypothesis_Test(Principal_Cells.Total_Surface, Interneuron_Cells.Total_Surface))

Significant Difference in Number of Branches:  True
Significant Difference in Volume:  True
Significant Difference in Surface:  True


In [8]:
#For the next 3 cells, we compare all the types of secondary class cells and see if there is a signifigant difference between the
#two being compared. Note: the order of the comparison does not affect the result, i.e. A compared to B is the same as B compared
# to A. As a result, our loops ony do one comparison be combination
Types = df.Secondary_Cell_Class.unique()
for n in range(0, len(Types)):
    data1 = df.where(df.Secondary_Cell_Class == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = df.where(df.Secondary_Cell_Class == Types[m]).dropna()
        #print('Significant differences in Number of Branches', Types[n],' ', Types[m],' : ',Hypothesis_Test(data1.Number_Branches,data2.Number_Branches))
        if((Hypothesis_Test(data1.Number_Branches,data2.Number_Branches))):
            print('Significant differences in Number of Branches', Types[n],' ', Types[m])
        else: 
            print('No Significant differences in Number of Branches', Types[n],' ', Types[m])

Significant differences in Number of Branches Not reported   pyramidal
Significant differences in Number of Branches Not reported   Neuropeptide Y (NPY)-positive
Significant differences in Number of Branches Not reported   Somatostatin (SOM) containing cell
No Significant differences in Number of Branches Not reported   ganglion
Significant differences in Number of Branches Not reported   Martinotti
Significant differences in Number of Branches Not reported   Parvalbumin (PV)-positive
Significant differences in Number of Branches Not reported   basket
No Significant differences in Number of Branches Not reported   vertical
Significant differences in Number of Branches Not reported   Schaffer-collateral associated
Significant differences in Number of Branches Not reported   Golgi
Significant differences in Number of Branches Not reported   granule
Significant differences in Number of Branches Not reported   medium spiny
Significant differences in Number of Branches Not reported   double

Significant differences in Number of Branches pyramidal   von Economo neuron
Significant differences in Number of Branches pyramidal   Chandelier
Significant differences in Number of Branches pyramidal   neurogliaform
Significant differences in Number of Branches pyramidal   semilunar granule
Significant differences in Number of Branches pyramidal   Purkinje
Significant differences in Number of Branches pyramidal   bipolar
No Significant differences in Number of Branches pyramidal   glutamatergic
Significant differences in Number of Branches pyramidal   Cajal-Retzius
Significant differences in Number of Branches pyramidal   Pyramidal-like
Significant differences in Number of Branches pyramidal   Fast-spiking
Significant differences in Number of Branches pyramidal   Non-glutamatergic
Significant differences in Number of Branches pyramidal   dendritic targeting
Significant differences in Number of Branches pyramidal   Motoneuron
Significant differences in Number of Branches pyramidal   M

Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   Multidendritic-dendritic arborization (DA)
Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   Aspiny
Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   Crab-like
Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   Lugaro
Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   sternzelle
No Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   bistratified
Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   GABAergic
Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   Muscarinic-responsive
No Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   perisomatic targeting
Significant differences in Number of Branches Neuropeptide Y (NPY)-positive   Neuroblast
Significant differences in Number of Branches Neuropeptide

Significant differences in Number of Branches Somatostatin (SOM) containing cell   Motoneuron
Significant differences in Number of Branches Somatostatin (SOM) containing cell   Multidendritic-dendritic arborization (DA)
Significant differences in Number of Branches Somatostatin (SOM) containing cell   Aspiny
Significant differences in Number of Branches Somatostatin (SOM) containing cell   Crab-like
Significant differences in Number of Branches Somatostatin (SOM) containing cell   Lugaro
Significant differences in Number of Branches Somatostatin (SOM) containing cell   sternzelle
No Significant differences in Number of Branches Somatostatin (SOM) containing cell   bistratified
Significant differences in Number of Branches Somatostatin (SOM) containing cell   GABAergic
Significant differences in Number of Branches Somatostatin (SOM) containing cell   Muscarinic-responsive
No Significant differences in Number of Branches Somatostatin (SOM) containing cell   perisomatic targeting
Signific

Significant differences in Number of Branches ganglion   Multidendritic-dendritic arborization (DA)
Significant differences in Number of Branches ganglion   Aspiny
Significant differences in Number of Branches ganglion   Crab-like
Significant differences in Number of Branches ganglion   Lugaro
Significant differences in Number of Branches ganglion   sternzelle
Significant differences in Number of Branches ganglion   bistratified
Significant differences in Number of Branches ganglion   GABAergic
No Significant differences in Number of Branches ganglion   Muscarinic-responsive
Significant differences in Number of Branches ganglion   perisomatic targeting
Significant differences in Number of Branches ganglion   Neuroblast
Significant differences in Number of Branches ganglion   somatic
Significant differences in Number of Branches ganglion   Ring
Significant differences in Number of Branches ganglion   Amphid
Significant differences in Number of Branches ganglion   multipolar
Significant 

...

No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   tripolar
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   cone
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   Type I
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   Type II
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   Intersubnuclear neuron
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   Ring/Pharynx
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   Nitrergic
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   Renshaw
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   axonless
No Significant differences in Number of Branches Cannabinoid receptor (CB1R)-positive   Small
No Significant differe

Significant differences in Number of Branches Pharyngeal   Nitrergic
Significant differences in Number of Branches Pharyngeal   Renshaw
Significant differences in Number of Branches Pharyngeal   axonless
Significant differences in Number of Branches Pharyngeal   Small
Significant differences in Number of Branches Pharyngeal   Descending
No Significant differences in Number of Branches Pharyngeal   Shepherd's crook neuron
No Significant differences in Number of Branches Pharyngeal   amacrine
Significant differences in Number of Branches Pharyngeal   modulated
Significant differences in Number of Branches Pharyngeal   Calbindin (CB)-positive
Significant differences in Number of Branches Pharyngeal   Cholecystokinin (CCK)-positive
Significant differences in Number of Branches Pharyngeal   Calretinin (CR)-positive
Significant differences in Number of Branches Pharyngeal   Total molecular layer projecting
No Significant differences in Number of Branches Pharyngeal   Nonpyramidal
No Signific

Significant differences in Number of Branches Local circuit neuron   Ring/Pharynx
Significant differences in Number of Branches Local circuit neuron   Nitrergic
Significant differences in Number of Branches Local circuit neuron   Renshaw
Significant differences in Number of Branches Local circuit neuron   axonless
Significant differences in Number of Branches Local circuit neuron   Small
Significant differences in Number of Branches Local circuit neuron   Descending
No Significant differences in Number of Branches Local circuit neuron   Shepherd's crook neuron
No Significant differences in Number of Branches Local circuit neuron   amacrine
Significant differences in Number of Branches Local circuit neuron   modulated
Significant differences in Number of Branches Local circuit neuron   Calbindin (CB)-positive
Significant differences in Number of Branches Local circuit neuron   Cholecystokinin (CCK)-positive
Significant differences in Number of Branches Local circuit neuron   Calretinin 

Significant differences in Number of Branches External tufted cell (ETC)   Small
No Significant differences in Number of Branches External tufted cell (ETC)   Descending
No Significant differences in Number of Branches External tufted cell (ETC)   Shepherd's crook neuron
No Significant differences in Number of Branches External tufted cell (ETC)   amacrine
Significant differences in Number of Branches External tufted cell (ETC)   modulated
Significant differences in Number of Branches External tufted cell (ETC)   Calbindin (CB)-positive
Significant differences in Number of Branches External tufted cell (ETC)   Cholecystokinin (CCK)-positive
Significant differences in Number of Branches External tufted cell (ETC)   Calretinin (CR)-positive
Significant differences in Number of Branches External tufted cell (ETC)   Total molecular layer projecting
No Significant differences in Number of Branches External tufted cell (ETC)   Nonpyramidal
No Significant differences in Number of Branches Ext

No Significant differences in Number of Branches Shepherd's crook neuron   Calbindin (CB)-positive
No Significant differences in Number of Branches Shepherd's crook neuron   Cholecystokinin (CCK)-positive
No Significant differences in Number of Branches Shepherd's crook neuron   Calretinin (CR)-positive
No Significant differences in Number of Branches Shepherd's crook neuron   Total molecular layer projecting
No Significant differences in Number of Branches Shepherd's crook neuron   Nonpyramidal
No Significant differences in Number of Branches Shepherd's crook neuron   interneuron-specific interneuron
No Significant differences in Number of Branches Shepherd's crook neuron   Spiny
No Significant differences in Number of Branches Shepherd's crook neuron   bursting
No Significant differences in Number of Branches amacrine   modulated
No Significant differences in Number of Branches amacrine   Calbindin (CB)-positive
No Significant differences in Number of Branches amacrine   Cholecystoki

In [9]:
Types = df.Secondary_Cell_Class.unique()
for n in range(0, len(Types)):
    data1 = df.where(df.Secondary_Cell_Class == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = df.where(df.Secondary_Cell_Class == Types[m]).dropna()
        #print('Significant differences in Volume', Types[n],' ', Types[m],' : ',Hypothesis_Test(data1.Total_Volume,data2.Total_Volume))
        if(Hypothesis_Test(data1.Total_Volume,data2.Total_Volume)):
            print('Significant differences in Volume', Types[n],' ', Types[m])
        else:
            print('No Significant differences in Volume', Types[n],' ', Types[m])

Significant differences in Volume Not reported   pyramidal
No Significant differences in Volume Not reported   Neuropeptide Y (NPY)-positive
No Significant differences in Volume Not reported   Somatostatin (SOM) containing cell
Significant differences in Volume Not reported   ganglion
Significant differences in Volume Not reported   Martinotti
Significant differences in Volume Not reported   Parvalbumin (PV)-positive
Significant differences in Volume Not reported   basket
Significant differences in Volume Not reported   vertical
Significant differences in Volume Not reported   Schaffer-collateral associated
Significant differences in Volume Not reported   Golgi
No Significant differences in Volume Not reported   granule
No Significant differences in Volume Not reported   medium spiny
No Significant differences in Volume Not reported   double bouquet
No Significant differences in Volume Not reported   thalamocortical
Significant differences in Volume Not reported   stellate
Significant 

Significant differences in Volume pyramidal   perisomatic targeting
Significant differences in Volume pyramidal   Neuroblast
Significant differences in Volume pyramidal   somatic
Significant differences in Volume pyramidal   Ring
Significant differences in Volume pyramidal   Amphid
Significant differences in Volume pyramidal   multipolar
Significant differences in Volume pyramidal   inverted
Significant differences in Volume pyramidal   tangential
Significant differences in Volume pyramidal   Horizontal
Significant differences in Volume pyramidal   Non-basket
Significant differences in Volume pyramidal   bitufted
Significant differences in Volume pyramidal   Non-fast spiking
No Significant differences in Volume pyramidal   inspiratory
Significant differences in Volume pyramidal   Choline acetyltransferase (ChAT)-positive
No Significant differences in Volume pyramidal   dopaminergic
Significant differences in Volume pyramidal   Trilaminar
Significant differences in Volume pyramidal   Pe

No Significant differences in Volume Neuropeptide Y (NPY)-positive   Non-cholinergic
No Significant differences in Volume Neuropeptide Y (NPY)-positive   cholinergic
Significant differences in Volume Neuropeptide Y (NPY)-positive   Pharyngeal
Significant differences in Volume Neuropeptide Y (NPY)-positive   mitral
No Significant differences in Volume Neuropeptide Y (NPY)-positive   tufted
Significant differences in Volume Neuropeptide Y (NPY)-positive   Ia inhibitory
Significant differences in Volume Neuropeptide Y (NPY)-positive   Local circuit neuron
Significant differences in Volume Neuropeptide Y (NPY)-positive   Projection neuron
Significant differences in Volume Neuropeptide Y (NPY)-positive   periglomerular
No Significant differences in Volume Neuropeptide Y (NPY)-positive   deep short axon
No Significant differences in Volume Neuropeptide Y (NPY)-positive   External tufted cell (ETC)
Significant differences in Volume Neuropeptide Y (NPY)-positive   olfactory
No Significant diff

No Significant differences in Volume Somatostatin (SOM) containing cell   deep short axon
No Significant differences in Volume Somatostatin (SOM) containing cell   External tufted cell (ETC)
Significant differences in Volume Somatostatin (SOM) containing cell   olfactory
No Significant differences in Volume Somatostatin (SOM) containing cell   tripolar
Significant differences in Volume Somatostatin (SOM) containing cell   cone
Significant differences in Volume Somatostatin (SOM) containing cell   Type I
Significant differences in Volume Somatostatin (SOM) containing cell   Type II
Significant differences in Volume Somatostatin (SOM) containing cell   Intersubnuclear neuron
Significant differences in Volume Somatostatin (SOM) containing cell   Ring/Pharynx
Significant differences in Volume Somatostatin (SOM) containing cell   Nitrergic
Significant differences in Volume Somatostatin (SOM) containing cell   Renshaw
Significant differences in Volume Somatostatin (SOM) containing cell   axo

Significant differences in Volume Martinotti   tyrosine-hydroxylase-positive
No Significant differences in Volume Martinotti   von Economo neuron
No Significant differences in Volume Martinotti   Chandelier
Significant differences in Volume Martinotti   neurogliaform
Significant differences in Volume Martinotti   semilunar granule
No Significant differences in Volume Martinotti   Purkinje
Significant differences in Volume Martinotti   bipolar
No Significant differences in Volume Martinotti   glutamatergic
Significant differences in Volume Martinotti   Cajal-Retzius
Significant differences in Volume Martinotti   Pyramidal-like
Significant differences in Volume Martinotti   Fast-spiking
Significant differences in Volume Martinotti   Non-glutamatergic
Significant differences in Volume Martinotti   dendritic targeting
Significant differences in Volume Martinotti   Motoneuron
Significant differences in Volume Martinotti   Multidendritic-dendritic arborization (DA)
Significant differences in

...

No Significant differences in Volume Non-cholinergic   tufted
Significant differences in Volume Non-cholinergic   Ia inhibitory
No Significant differences in Volume Non-cholinergic   Local circuit neuron
No Significant differences in Volume Non-cholinergic   Projection neuron
Significant differences in Volume Non-cholinergic   periglomerular
No Significant differences in Volume Non-cholinergic   deep short axon
No Significant differences in Volume Non-cholinergic   External tufted cell (ETC)
No Significant differences in Volume Non-cholinergic   olfactory
No Significant differences in Volume Non-cholinergic   tripolar
Significant differences in Volume Non-cholinergic   cone
Significant differences in Volume Non-cholinergic   Type I
Significant differences in Volume Non-cholinergic   Type II
Significant differences in Volume Non-cholinergic   Intersubnuclear neuron
Significant differences in Volume Non-cholinergic   Ring/Pharynx
Significant differences in Volume Non-cholinergic   Nitrer

No Significant differences in Volume tufted   External tufted cell (ETC)
Significant differences in Volume tufted   olfactory
No Significant differences in Volume tufted   tripolar
Significant differences in Volume tufted   cone
Significant differences in Volume tufted   Type I
Significant differences in Volume tufted   Type II
Significant differences in Volume tufted   Intersubnuclear neuron
Significant differences in Volume tufted   Ring/Pharynx
Significant differences in Volume tufted   Nitrergic
Significant differences in Volume tufted   Renshaw
Significant differences in Volume tufted   axonless
Significant differences in Volume tufted   Small
No Significant differences in Volume tufted   Descending
No Significant differences in Volume tufted   Shepherd's crook neuron
No Significant differences in Volume tufted   amacrine
Significant differences in Volume tufted   modulated
No Significant differences in Volume tufted   Calbindin (CB)-positive
Significant differences in Volume tuft

No Significant differences in Volume periglomerular   Nonpyramidal
No Significant differences in Volume periglomerular   interneuron-specific interneuron
No Significant differences in Volume periglomerular   Spiny
No Significant differences in Volume periglomerular   bursting
No Significant differences in Volume deep short axon   External tufted cell (ETC)
Significant differences in Volume deep short axon   olfactory
Significant differences in Volume deep short axon   tripolar
Significant differences in Volume deep short axon   cone
Significant differences in Volume deep short axon   Type I
Significant differences in Volume deep short axon   Type II
Significant differences in Volume deep short axon   Intersubnuclear neuron
Significant differences in Volume deep short axon   Ring/Pharynx
Significant differences in Volume deep short axon   Nitrergic
Significant differences in Volume deep short axon   Renshaw
Significant differences in Volume deep short axon   axonless
Significant differe

No Significant differences in Volume Type I   Nonpyramidal
No Significant differences in Volume Type I   interneuron-specific interneuron
No Significant differences in Volume Type I   Spiny
No Significant differences in Volume Type I   bursting
No Significant differences in Volume Type II   Intersubnuclear neuron
Significant differences in Volume Type II   Ring/Pharynx
No Significant differences in Volume Type II   Nitrergic
Significant differences in Volume Type II   Renshaw
Significant differences in Volume Type II   axonless
No Significant differences in Volume Type II   Small
No Significant differences in Volume Type II   Descending
No Significant differences in Volume Type II   Shepherd's crook neuron
No Significant differences in Volume Type II   amacrine
Significant differences in Volume Type II   modulated
Significant differences in Volume Type II   Calbindin (CB)-positive
Significant differences in Volume Type II   Cholecystokinin (CCK)-positive
No Significant differences in V

No Significant differences in Volume Shepherd's crook neuron   Nonpyramidal
No Significant differences in Volume Shepherd's crook neuron   interneuron-specific interneuron
No Significant differences in Volume Shepherd's crook neuron   Spiny
No Significant differences in Volume Shepherd's crook neuron   bursting
No Significant differences in Volume amacrine   modulated
No Significant differences in Volume amacrine   Calbindin (CB)-positive
No Significant differences in Volume amacrine   Cholecystokinin (CCK)-positive
No Significant differences in Volume amacrine   Calretinin (CR)-positive
No Significant differences in Volume amacrine   Total molecular layer projecting
No Significant differences in Volume amacrine   Nonpyramidal
No Significant differences in Volume amacrine   interneuron-specific interneuron
No Significant differences in Volume amacrine   Spiny
No Significant differences in Volume amacrine   bursting
Significant differences in Volume modulated   Calbindin (CB)-positive
S

In [10]:
Types = df.Secondary_Cell_Class.unique()
for n in range(0, len(Types)):
    data1 = df.where(df.Secondary_Cell_Class == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = df.where(df.Secondary_Cell_Class == Types[m]).dropna()
        #print('Significant differences in Surface', Types[n],' ', Types[m],' : ',Hypothesis_Test(data1.Total_Surface,data2.Total_Surface))
        if(Hypothesis_Test(data1.Total_Surface,data2.Total_Surface)):
            print('Significant differences in Surface', Types[n],' ', Types[m])
        else:
            print('No Significant differences in Surface', Types[n],' ', Types[m])

Significant differences in Surface Not reported   pyramidal
Significant differences in Surface Not reported   Neuropeptide Y (NPY)-positive
Significant differences in Surface Not reported   Somatostatin (SOM) containing cell
Significant differences in Surface Not reported   ganglion
Significant differences in Surface Not reported   Martinotti
Significant differences in Surface Not reported   Parvalbumin (PV)-positive
Significant differences in Surface Not reported   basket
Significant differences in Surface Not reported   vertical
Significant differences in Surface Not reported   Schaffer-collateral associated
Significant differences in Surface Not reported   Golgi
Significant differences in Surface Not reported   granule
Significant differences in Surface Not reported   medium spiny
Significant differences in Surface Not reported   double bouquet
No Significant differences in Surface Not reported   thalamocortical
Significant differences in Surface Not reported   stellate
Significant 

Significant differences in Surface pyramidal   perisomatic targeting
Significant differences in Surface pyramidal   Neuroblast
Significant differences in Surface pyramidal   somatic
Significant differences in Surface pyramidal   Ring
Significant differences in Surface pyramidal   Amphid
No Significant differences in Surface pyramidal   multipolar
No Significant differences in Surface pyramidal   inverted
Significant differences in Surface pyramidal   tangential
No Significant differences in Surface pyramidal   Horizontal
Significant differences in Surface pyramidal   Non-basket
Significant differences in Surface pyramidal   bitufted
No Significant differences in Surface pyramidal   Non-fast spiking
Significant differences in Surface pyramidal   inspiratory
Significant differences in Surface pyramidal   Choline acetyltransferase (ChAT)-positive
Significant differences in Surface pyramidal   dopaminergic
Significant differences in Surface pyramidal   Trilaminar
Significant differences in

No Significant differences in Surface Neuropeptide Y (NPY)-positive   cholinergic
Significant differences in Surface Neuropeptide Y (NPY)-positive   Pharyngeal
No Significant differences in Surface Neuropeptide Y (NPY)-positive   mitral
Significant differences in Surface Neuropeptide Y (NPY)-positive   tufted
Significant differences in Surface Neuropeptide Y (NPY)-positive   Ia inhibitory
Significant differences in Surface Neuropeptide Y (NPY)-positive   Local circuit neuron
No Significant differences in Surface Neuropeptide Y (NPY)-positive   Projection neuron
Significant differences in Surface Neuropeptide Y (NPY)-positive   periglomerular
No Significant differences in Surface Neuropeptide Y (NPY)-positive   deep short axon
Significant differences in Surface Neuropeptide Y (NPY)-positive   External tufted cell (ETC)
No Significant differences in Surface Neuropeptide Y (NPY)-positive   olfactory
Significant differences in Surface Neuropeptide Y (NPY)-positive   tripolar
Significant di

No Significant differences in Surface Somatostatin (SOM) containing cell   External tufted cell (ETC)
No Significant differences in Surface Somatostatin (SOM) containing cell   olfactory
Significant differences in Surface Somatostatin (SOM) containing cell   tripolar
Significant differences in Surface Somatostatin (SOM) containing cell   cone
Significant differences in Surface Somatostatin (SOM) containing cell   Type I
Significant differences in Surface Somatostatin (SOM) containing cell   Type II
Significant differences in Surface Somatostatin (SOM) containing cell   Intersubnuclear neuron
Significant differences in Surface Somatostatin (SOM) containing cell   Ring/Pharynx
Significant differences in Surface Somatostatin (SOM) containing cell   Nitrergic
No Significant differences in Surface Somatostatin (SOM) containing cell   Renshaw
Significant differences in Surface Somatostatin (SOM) containing cell   axonless
Significant differences in Surface Somatostatin (SOM) containing cell 

Significant differences in Surface Martinotti   tyrosine-hydroxylase-positive
Significant differences in Surface Martinotti   von Economo neuron
Significant differences in Surface Martinotti   Chandelier
Significant differences in Surface Martinotti   neurogliaform
Significant differences in Surface Martinotti   semilunar granule
No Significant differences in Surface Martinotti   Purkinje
Significant differences in Surface Martinotti   bipolar
No Significant differences in Surface Martinotti   glutamatergic
Significant differences in Surface Martinotti   Cajal-Retzius
Significant differences in Surface Martinotti   Pyramidal-like
Significant differences in Surface Martinotti   Fast-spiking
Significant differences in Surface Martinotti   Non-glutamatergic
Significant differences in Surface Martinotti   dendritic targeting
Significant differences in Surface Martinotti   Motoneuron
Significant differences in Surface Martinotti   Multidendritic-dendritic arborization (DA)
Significant diffe

...

No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   Projection neuron
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   periglomerular
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   deep short axon
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   External tufted cell (ETC)
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   olfactory
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   tripolar
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   cone
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   Type I
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   Type II
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   Intersubnuclear neuron
No Significant differences in Surface Cannabinoid receptor (CB1R)-positive   Ring/Pharynx
No

Significant differences in Surface Pharyngeal   Calretinin (CR)-positive
Significant differences in Surface Pharyngeal   Total molecular layer projecting
No Significant differences in Surface Pharyngeal   Nonpyramidal
No Significant differences in Surface Pharyngeal   interneuron-specific interneuron
No Significant differences in Surface Pharyngeal   Spiny
No Significant differences in Surface Pharyngeal   bursting
Significant differences in Surface mitral   tufted
Significant differences in Surface mitral   Ia inhibitory
Significant differences in Surface mitral   Local circuit neuron
Significant differences in Surface mitral   Projection neuron
Significant differences in Surface mitral   periglomerular
No Significant differences in Surface mitral   deep short axon
Significant differences in Surface mitral   External tufted cell (ETC)
No Significant differences in Surface mitral   olfactory
Significant differences in Surface mitral   tripolar
Significant differences in Surface mitral 

Significant differences in Surface Projection neuron   cone
Significant differences in Surface Projection neuron   Type I
Significant differences in Surface Projection neuron   Type II
Significant differences in Surface Projection neuron   Intersubnuclear neuron
Significant differences in Surface Projection neuron   Ring/Pharynx
Significant differences in Surface Projection neuron   Nitrergic
No Significant differences in Surface Projection neuron   Renshaw
Significant differences in Surface Projection neuron   axonless
Significant differences in Surface Projection neuron   Small
Significant differences in Surface Projection neuron   Descending
No Significant differences in Surface Projection neuron   Shepherd's crook neuron
No Significant differences in Surface Projection neuron   amacrine
No Significant differences in Surface Projection neuron   modulated
Significant differences in Surface Projection neuron   Calbindin (CB)-positive
Significant differences in Surface Projection neuro

Significant differences in Surface tripolar   Nitrergic
Significant differences in Surface tripolar   Renshaw
Significant differences in Surface tripolar   axonless
No Significant differences in Surface tripolar   Small
Significant differences in Surface tripolar   Descending
No Significant differences in Surface tripolar   Shepherd's crook neuron
No Significant differences in Surface tripolar   amacrine
Significant differences in Surface tripolar   modulated
Significant differences in Surface tripolar   Calbindin (CB)-positive
Significant differences in Surface tripolar   Cholecystokinin (CCK)-positive
No Significant differences in Surface tripolar   Calretinin (CR)-positive
Significant differences in Surface tripolar   Total molecular layer projecting
No Significant differences in Surface tripolar   Nonpyramidal
No Significant differences in Surface tripolar   interneuron-specific interneuron
No Significant differences in Surface tripolar   Spiny
No Significant differences in Surface

No Significant differences in Surface Renshaw   Cholecystokinin (CCK)-positive
Significant differences in Surface Renshaw   Calretinin (CR)-positive
Significant differences in Surface Renshaw   Total molecular layer projecting
No Significant differences in Surface Renshaw   Nonpyramidal
No Significant differences in Surface Renshaw   interneuron-specific interneuron
No Significant differences in Surface Renshaw   Spiny
No Significant differences in Surface Renshaw   bursting
Significant differences in Surface axonless   Small
Significant differences in Surface axonless   Descending
No Significant differences in Surface axonless   Shepherd's crook neuron
No Significant differences in Surface axonless   amacrine
Significant differences in Surface axonless   modulated
Significant differences in Surface axonless   Calbindin (CB)-positive
Significant differences in Surface axonless   Cholecystokinin (CCK)-positive
Significant differences in Surface axonless   Calretinin (CR)-positive
Signif

In [40]:
#Since running through ~6000 results for one comparison, we defined a compare function for all three qualities we were hunting
def compare_secondary_branches(type1,type2,df):
    data1 = df.where(df.Secondary_Cell_Class == type1).dropna()
    data2 = df.where(df.Secondary_Cell_Class == type2).dropna()
    if(Hypothesis_Test(data1.Number_Branches,data2.Number_Branches)):
            print('Significant differences in Branches' , type1,' ', type2)
    else:
            print('No Significant differences in Branches', type1,' ', type2)
def compare_secondary_volume(type1,type2,df):
    data1 = df.where(df.Secondary_Cell_Class == type1).dropna()
    data2 = df.where(df.Secondary_Cell_Class == type2).dropna()
    if(Hypothesis_Test(data1.Total_Volume,data2.Total_Volume)):
            print('Significant differences in Volume' , type1,' ', type2)
    else:
            print('No Significant differences in Volume', type1,' ', type2) 
def compare_secondary_surface(type1,type2,df):
    data1 = df.where(df.Secondary_Cell_Class == type1).dropna()
    data2 = df.where(df.Secondary_Cell_Class == type2).dropna()
    if(Hypothesis_Test(data1.Total_Surface,data2.Total_Surface)):
            print('Significant differences in Surface' , type1,' ', type2)
    else:
            print('No Significant differences in Surface', type1,' ', type2) 

In [46]:
#Now we set up to determine if any staining, slicing direction, or thickness provide less data than the others. First we find the
#amount of fields nor reported in the data and make a new dataframe that contains the amount of not reports and the stain, slicing 
#direction, and sliing thickness
not_reported_np = np.zeros(shape=(len(df),4))
not_reported = pd.DataFrame(not_reported_np)
for n in range(0, len(df)):
    number_not_reported = 0
    for m in range(0,53):
        if(df.iat[n,m]=='Not reported'):
            number_not_reported += 1
    not_reported.set_value(n,0,number_not_reported)
not_reported[1]=df.Staining_Method
not_reported[2]=df.Slicing_Direction
not_reported[3]=df.Slice_Thickness
not_reported.columns = ['Number_not_reported', 'Stain','Slice_Direction', 'Slice_Thickness']
not_reported = Sort('Stain', not_reported)

In [54]:
Types = df.Staining_Method.unique()
#We now make a dataframe witht the stain, average amount not reported, and the standard deviation of the not reported. We then take
#the minimum in order to find the stain with the least non-reports
Averages_np = np.zeros(shape=(len(Types),3))
Averages_stain = pd.DataFrame(Averages_np)
for n in range(0,len(Types)):
    data = not_reported.where(not_reported.Stain == Types[n])
    Averages_stain.set_value(n,0,data.mean())
    Averages_stain.set_value(n,1,data.std())
Averages_stain[2]=Types
Averages_stain.columns = ['Average', 'Standard_Deviation', 'Stain']


In [57]:
Minimum_stain = Averages.where(Averages.Average ==Averages.Average.min()).dropna()
Minimum_stain
#So biotinylated dextran amine gives the least amount of non-reports

,Average,Standard_Deviation,Stain
32,1.674419,3.273573,biotinylated dextran amine


In [13]:
for n in range(0, len(Types)):
    data1 = not_reported.where(not_reported.Stain == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = not_reported.where(not_reported.Stain == Types[m]).dropna()
        if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences amount reported', Types[n],' ', Types[m])
        else:
            print('No Significant differences in amounr reported', Types[n],' ', Types[m])

Significant differences amount reported neurobiotin   biocytin
Significant differences amount reported neurobiotin   green fluorescent protein
Significant differences amount reported neurobiotin   Multiple
Significant differences amount reported neurobiotin   red fluorescent protein
Significant differences amount reported neurobiotin   Golgi
Significant differences amount reported neurobiotin   Not reported
Significant differences amount reported neurobiotin   NMDA-BDA
No Significant differences in amounr reported neurobiotin   Alexa Fluor 594
No Significant differences in amounr reported neurobiotin   lucifer yellow
No Significant differences in amounr reported neurobiotin   Alexa Fluor 488
Significant differences amount reported neurobiotin   Sindbis palGFP virus
Significant differences amount reported neurobiotin   Golgi-Cox
Significant differences amount reported neurobiotin   enhanced green fluorescent protein
Significant differences amount reported neurobiotin   horseradish perox

Significant differences amount reported Multiple   NMDA-BDA
Significant differences amount reported Multiple   Alexa Fluor 594
Significant differences amount reported Multiple   lucifer yellow
Significant differences amount reported Multiple   Alexa Fluor 488
Significant differences amount reported Multiple   Sindbis palGFP virus
Significant differences amount reported Multiple   Golgi-Cox
Significant differences amount reported Multiple   enhanced green fluorescent protein
Significant differences amount reported Multiple   horseradish peroxidase
Significant differences amount reported Multiple   Rapid Golgi
Significant differences amount reported Multiple   Biocytin, Streptavidin-Alexa 546
Significant differences amount reported Multiple   Golgi-Scheibel
Significant differences amount reported Multiple   DiI
Significant differences amount reported Multiple   immunostaining
Significant differences amount reported Multiple   AlexaFluor 488
Significant differences amount reported Multipl

No Significant differences in amounr reported Not reported   biotinamide
Significant differences amount reported Not reported   turboRFP
Significant differences amount reported Not reported   dextran coupled texas red fluorophore
Significant differences amount reported Not reported   Green fluorescent protein (GFP), Alexa Fluor 594 hydrazide
Significant differences amount reported Not reported   biotinylated dextran amine
Significant differences amount reported Not reported   NADPH-diaphorase histochesmistry
No Significant differences in amounr reported Not reported   Farnesylated enhanced green fluorescent protein (eGFP-F)
No Significant differences in amounr reported Not reported   myristoylated-RFP
Significant differences amount reported Not reported   DiOlistics
Significant differences amount reported Not reported   yellow fluorescent protein
Significant differences amount reported NMDA-BDA   Alexa Fluor 594
Significant differences amount reported NMDA-BDA   lucifer yellow
Signific

Significant differences amount reported Alexa Fluor 488   biotinylated dextran amine
No Significant differences in amounr reported Alexa Fluor 488   NADPH-diaphorase histochesmistry
No Significant differences in amounr reported Alexa Fluor 488   Farnesylated enhanced green fluorescent protein (eGFP-F)
No Significant differences in amounr reported Alexa Fluor 488   myristoylated-RFP
Significant differences amount reported Alexa Fluor 488   DiOlistics
No Significant differences in amounr reported Alexa Fluor 488   yellow fluorescent protein
Significant differences amount reported Sindbis palGFP virus   Golgi-Cox
Significant differences amount reported Sindbis palGFP virus   enhanced green fluorescent protein
Significant differences amount reported Sindbis palGFP virus   horseradish peroxidase
Significant differences amount reported Sindbis palGFP virus   Rapid Golgi
Significant differences amount reported Sindbis palGFP virus   Biocytin, Streptavidin-Alexa 546
Significant differences amo

Significant differences amount reported horseradish peroxidase   dextran coupled texas red fluorophore
Significant differences amount reported horseradish peroxidase   Green fluorescent protein (GFP), Alexa Fluor 594 hydrazide
Significant differences amount reported horseradish peroxidase   biotinylated dextran amine
Significant differences amount reported horseradish peroxidase   NADPH-diaphorase histochesmistry
No Significant differences in amounr reported horseradish peroxidase   Farnesylated enhanced green fluorescent protein (eGFP-F)
No Significant differences in amounr reported horseradish peroxidase   myristoylated-RFP
Significant differences amount reported horseradish peroxidase   DiOlistics
Significant differences amount reported horseradish peroxidase   yellow fluorescent protein
Significant differences amount reported Rapid Golgi   Biocytin, Streptavidin-Alexa 546
Significant differences amount reported Rapid Golgi   Golgi-Scheibel
Significant differences amount reported Ra

Significant differences amount reported immunostaining   turboRFP
Significant differences amount reported immunostaining   dextran coupled texas red fluorophore
Significant differences amount reported immunostaining   Green fluorescent protein (GFP), Alexa Fluor 594 hydrazide
Significant differences amount reported immunostaining   biotinylated dextran amine
Significant differences amount reported immunostaining   NADPH-diaphorase histochesmistry
No Significant differences in amounr reported immunostaining   Farnesylated enhanced green fluorescent protein (eGFP-F)
No Significant differences in amounr reported immunostaining   myristoylated-RFP
Significant differences amount reported immunostaining   DiOlistics
Significant differences amount reported immunostaining   yellow fluorescent protein
Significant differences amount reported AlexaFluor 488   alkaline phosphatase
Significant differences amount reported AlexaFluor 488   Alexa Fluor 594 hydrazide
Significant differences amount repo

No Significant differences in amounr reported Alexa 568 hydrazide   Farnesylated enhanced green fluorescent protein (eGFP-F)
No Significant differences in amounr reported Alexa 568 hydrazide   myristoylated-RFP
Significant differences amount reported Alexa 568 hydrazide   DiOlistics
Significant differences amount reported Alexa 568 hydrazide   yellow fluorescent protein
Significant differences amount reported cobalt sulphide   Mag-indo-1 AM
No Significant differences in amounr reported cobalt sulphide   biotinamide
Significant differences amount reported cobalt sulphide   turboRFP
Significant differences amount reported cobalt sulphide   dextran coupled texas red fluorophore
Significant differences amount reported cobalt sulphide   Green fluorescent protein (GFP), Alexa Fluor 594 hydrazide
Significant differences amount reported cobalt sulphide   biotinylated dextran amine
Significant differences amount reported cobalt sulphide   NADPH-diaphorase histochesmistry
No Significant differen

In [74]:
#Again, since there are many results, we defince search functions in order to quickly find the comparison wanted
def compare_stain(type1,type2,df):
    data1 = df.where(df.Stain == type1).dropna()
    data2 = df.where(df.Stain == type2).dropna()
    if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences in number reported' , type1,' ', type2)
    else:
            print('No Significant differences in number reported', type1,' ', type2)

In [63]:
#We then apply this to the slicing direction and slicing thickness
Types = df.Slicing_Direction.unique()
Averages_np = np.zeros(shape=(len(Types),3))
Averages_slice_direction = pd.DataFrame(Averages_np)
for n in range(0,len(Types)):
    data = not_reported.where(not_reported.Slice_Direction == Types[n])
    Averages_slice_direction.set_value(n,0,data.mean())
    Averages_slice_direction.set_value(n,1,data.std())
Averages_slice_direction[2]=Types
Averages_slice_direction.columns = ['Average', 'Standard_Deviation', 'Slice_Direction']

In [64]:
Minimum_slicing_direction = Averages_slice_direction.where(Averages_slice_direction.Average ==Averages_slice_direction.Average.min()).dropna()
Minimum_slicing_direction
#So oblique horizontal cuts gives the least amount of non-reports

,Average,Standard_Deviation,Slice_Direction
9,2.84472,0.703198,oblique horizontal


In [15]:
for n in range(0, len(Types)):
    data1 = not_reported.where(not_reported.Slice_Direction == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = not_reported.where(not_reported.Slice_Direction == Types[m]).dropna()
        if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences amount reported', Types[n],' ', Types[m])
        else:
            print('No Significant differences in amount reported', Types[n],' ', Types[m])

No Significant differences in amounr reported horizontal   coronal
Significant differences amount reported horizontal   whole mount
Significant differences amount reported horizontal   Not reported
Significant differences amount reported horizontal   parasagittal
Significant differences amount reported horizontal   custom
No Significant differences in amounr reported horizontal   parallel to the cortical surface
Significant differences amount reported horizontal   thalamocortical
Significant differences amount reported horizontal   semi-coronal
Significant differences amount reported horizontal   oblique horizontal
Significant differences amount reported horizontal   tangential
Significant differences amount reported horizontal   near-coronal
Significant differences amount reported coronal   whole mount
Significant differences amount reported coronal   Not reported
Significant differences amount reported coronal   parasagittal
Significant differences amount reported coronal   custom
No

In [76]:
def compare_slicing_direction(type1,type2,df):
    data1 = df.where(df.Slice_Direction == type1).dropna()
    data2 = df.where(df.Slice_Direction == type2).dropna()
    if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences in number reported' , type1,' ', type2)
    else:
            print('No Significant differences in number reported', type1,' ', type2)

In [66]:
#We treat the thickness of the cuts as a categorical dataset becuse the values for the thickness is grouped into bins (caused by rounding)
#, and as a result, the accuracy of a linear regression is questionable
Types = df.Slice_Thickness.unique()
Averages_np = np.zeros(shape=(len(Types),3))
Averages_slice_thickness = pd.DataFrame(Averages_np)
for n in range(0,len(Types)):
    data = not_reported.where(not_reported.Slice_Thickness == Types[n])
    Averages_slice_thickness.set_value(n,0,data.mean())
    Averages_slice_thickness.set_value(n,1,data.std())
Averages_slice_thickness[2]=Types
Averages_slice_thickness.columns = ['Average', 'Standard_Deviation', 'Slice_Direction']

In [67]:
Minimum_slicing_thickness = Averages_slice_thickness.where(Averages_slice_thickness.Average ==Averages_slice_thickness.Average.min()).dropna()
Minimum_slicing_thickness
#So 65 thick cuts gives the least amount of non-reports

,Average,Standard_Deviation,Slice_Direction
24,1.411215,1.466309,65


In [17]:
for n in range(0, len(Types)):
    data1 = not_reported.where(not_reported.Slice_Thickness == Types[n]).dropna()
    for m in range(n+1, len(Types)):
        data2 = not_reported.where(not_reported.Slice_Thickness == Types[m]).dropna()
        if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences amount reported', Types[n],' ', Types[m])
        else:
            print('No Significant differences in amount reported', Types[n],' ', Types[m])

Significant differences amount reported 300   Not reported
Significant differences amount reported 300   100
Significant differences amount reported 300   400
No Significant differences in amounr reported 300   60
Significant differences amount reported 300   90
Significant differences amount reported 300   250
Significant differences amount reported 300   350
No Significant differences in amounr reported 300   50
Significant differences amount reported 300   200
No Significant differences in amounr reported 300   1000
Significant differences amount reported 300   150
Significant differences amount reported 300   120
Significant differences amount reported 300   40
Significant differences amount reported 300   550
No Significant differences in amounr reported 300   228
No Significant differences in amounr reported 300   178
Significant differences amount reported 300   225
No Significant differences in amounr reported 300   80
No Significant differences in amounr reported 300   88
No S

Significant differences amount reported 60   600
Significant differences amount reported 60   275
Significant differences amount reported 60   140
Significant differences amount reported 60   30
Significant differences amount reported 60   500
Significant differences amount reported 90   250
Significant differences amount reported 90   350
Significant differences amount reported 90   50
Significant differences amount reported 90   200
No Significant differences in amounr reported 90   1000
Significant differences amount reported 90   150
Significant differences amount reported 90   120
No Significant differences in amounr reported 90   40
Significant differences amount reported 90   550
No Significant differences in amounr reported 90   228
No Significant differences in amounr reported 90   178
Significant differences amount reported 90   225
No Significant differences in amounr reported 90   80
No Significant differences in amounr reported 90   88
No Significant differences in amounr 

Significant differences amount reported 150   550
No Significant differences in amounr reported 150   228
No Significant differences in amounr reported 150   178
Significant differences amount reported 150   225
Significant differences amount reported 150   80
No Significant differences in amounr reported 150   88
No Significant differences in amounr reported 150   85
Significant differences amount reported 150   75
Significant differences amount reported 150   70
No Significant differences in amounr reported 150   325
Significant differences amount reported 150   65
Significant differences amount reported 150   220
No Significant differences in amounr reported 150   234
Significant differences amount reported 150   305
No Significant differences in amounr reported 150   130
No Significant differences in amounr reported 150   294
Significant differences amount reported 150   600
Significant differences amount reported 150   275
Significant differences amount reported 150   140
Signific

No Significant differences in amounr reported 88   500
No Significant differences in amounr reported 85   75
No Significant differences in amounr reported 85   70
No Significant differences in amounr reported 85   325
No Significant differences in amounr reported 85   65
No Significant differences in amounr reported 85   220
No Significant differences in amounr reported 85   234
No Significant differences in amounr reported 85   305
No Significant differences in amounr reported 85   130
No Significant differences in amounr reported 85   294
No Significant differences in amounr reported 85   600
No Significant differences in amounr reported 85   275
No Significant differences in amounr reported 85   140
No Significant differences in amounr reported 85   30
No Significant differences in amounr reported 85   500
No Significant differences in amounr reported 75   70
No Significant differences in amounr reported 75   325
Significant differences amount reported 75   65
No Significant differe

In [89]:
def compare_slicing_thickness(type1,type2,df):
    data1 = df.where(df.Slice_Thickness == type1).dropna()
    data2 = df.where(df.Slice_Thickness == type2).dropna()
    if(Hypothesis_Test(data1.Number_not_reported,data2.Number_not_reported)):
            print('Significant differences in number reported' , type1,' ', type2)
    else:
            print('No Significant differences in number reported', type1,' ', type2)

In [9]:
Types = df.Species_Name.unique()
for n in range(0,len(Types)):
    data1 = df.where(df.Species_Name==Types[n])
    data1 = data1.where(data1.Primary_Brain_Region == 'hippocampus')
    for m in range(n+1,len(Types)):
        data2 = df.where(df.Species_Name==Types[m])
        data2 = data2.where(data2.Primary_Brain_Region == 'hippocampus')
        if(Hypothesis_Test(data1.Total_Volume,data2.Total_Volume)):
            print('Significant differences in Volume', Types[n],' ', Types[m])
        else:
            print('No Significant differences in Volume', Types[n],' ', Types[m])

No Significant differences in Volume rat   rat
Significant differences in Volume rat   mouse
No Significant differences in Volume rat   cat
No Significant differences in Volume rat   monkey
No Significant differences in Volume rat   human
No Significant differences in Volume rat   chicken
No Significant differences in Volume rat   chimpanzee
No Significant differences in Volume rat   frog
No Significant differences in Volume rat   drosophila melanogaster
No Significant differences in Volume rat   elephant
No Significant differences in Volume rat   minke whale
No Significant differences in Volume rat   bottlenose dolphin
No Significant differences in Volume rat   humpback whale
No Significant differences in Volume rat   giraffe
No Significant differences in Volume rat   manatee
No Significant differences in Volume rat   Siberian tiger
No Significant differences in Volume rat   clouded leopard
No Significant differences in Volume rat   goldfish
No Significant differences in Volume rat   

No Significant differences in Volume chicken   goldfish
No Significant differences in Volume chicken   zebrafish
No Significant differences in Volume chicken   guinea pig
No Significant differences in Volume chicken   sheep
No Significant differences in Volume chicken   C. elegans
No Significant differences in Volume chicken   proechimys
No Significant differences in Volume chicken   salamander
No Significant differences in Volume chicken   pouched lamprey
No Significant differences in Volume chicken   blowfly
No Significant differences in Volume chicken   moth
No Significant differences in Volume chicken   spiny lobster
No Significant differences in Volume chicken   domestic pig
No Significant differences in Volume chicken   rabbit
No Significant differences in Volume chimpanzee   chimpanzee
No Significant differences in Volume chimpanzee   frog
No Significant differences in Volume chimpanzee   drosophila melanogaster
No Significant differences in Volume chimpanzee   elephant
No Signi

No Significant differences in Volume bottlenose dolphin   sheep
No Significant differences in Volume bottlenose dolphin   C. elegans
No Significant differences in Volume bottlenose dolphin   proechimys
No Significant differences in Volume bottlenose dolphin   salamander
No Significant differences in Volume bottlenose dolphin   pouched lamprey
No Significant differences in Volume bottlenose dolphin   blowfly
No Significant differences in Volume bottlenose dolphin   moth
No Significant differences in Volume bottlenose dolphin   spiny lobster
No Significant differences in Volume bottlenose dolphin   domestic pig
No Significant differences in Volume bottlenose dolphin   rabbit
No Significant differences in Volume humpback whale   humpback whale
No Significant differences in Volume humpback whale   giraffe
No Significant differences in Volume humpback whale   manatee
No Significant differences in Volume humpback whale   Siberian tiger
No Significant differences in Volume humpback whale   cl

No Significant differences in Volume sheep   rabbit
No Significant differences in Volume C. elegans   C. elegans
No Significant differences in Volume C. elegans   proechimys
No Significant differences in Volume C. elegans   salamander
No Significant differences in Volume C. elegans   pouched lamprey
No Significant differences in Volume C. elegans   blowfly
No Significant differences in Volume C. elegans   moth
No Significant differences in Volume C. elegans   spiny lobster
No Significant differences in Volume C. elegans   domestic pig
No Significant differences in Volume C. elegans   rabbit
No Significant differences in Volume proechimys   proechimys
No Significant differences in Volume proechimys   salamander
No Significant differences in Volume proechimys   pouched lamprey
No Significant differences in Volume proechimys   blowfly
No Significant differences in Volume proechimys   moth
No Significant differences in Volume proechimys   spiny lobster
Significant differences in Volume pro

In [91]:
def compare_species(type1,type2,df):
    data1 = df.where(df.Species_Name == type1).dropna()
    data1 = data1.where(data1.Primary_Brain_Region == 'hippocampus')
    data2 = df.where(df.Species_Name == type2).dropna()
    data2 = data2.where(data2.Primary_Brain_Region == 'hippocampus')
    if(Hypothesis_Test(data1.Total_Volume,data2.Total_Volume)):
            print('Significant differences in Volume' , type1,' ', type2)
    else:
            print('No Significant differences in Volume', type1,' ', type2)

In [ ]:
# Classification function to determine cell class, brain region, and species name by using PCA on morphology data
def neuronClassifier(predictors, outputLabel):
    predictDf = df[predictors]
    
    # Standardization to account for different scales of the variables
    predictDf_standardized = StandardScaler().fit_transform(predictDf)

    outputVector = df[outputLabel]

    # PCA to determine the 3 principal components and reduce dimensionality
    sklearn_pca = PCA(n_components=3)
    Y_sklearn = sklearn_pca.fit_transform(predictDf_standardized)
    reducedDf = pd.DataFrame(Y_sklearn)
    
    # Split the data into testing and training data
    testingIndices = []
    for x in range(100):
        randonum = random.randint(0,len(df.index))
        while randonum in testingIndices:
            randonum = random.randint(0,len(df.index))
        testingIndices.append(randonum)

    trainingIndices = [x for x in range(0,len(df.index)) if x not in testingIndices]
    testingSet = reducedDf.iloc[testingIndices,:]
    trainingSet = reducedDf.iloc[trainingIndices,:]

    testLabels = list(outputVector[testingIndices])
    trainLabels = list(outputVector[trainingIndices])
    
    # Generate predictions via majority vote of the 15 nearest neighbors in the training set 
    predictionVector = []
    for i, testInd in enumerate(range(len(testingSet.index))):
        print(i)
        testRow = testingSet.iloc[testInd,:]
        distanceVector = []
        for trainInd in range(len(trainingSet.index)):
            trainRow = trainingSet.iloc[trainInd,:]
            distance = float(euclidean_distances(testRow, trainRow)[0][0])
            distanceVector.append((trainInd, distance))
        distanceVector = sorted(distanceVector, key = lambda x: x[1])
        nearestNeighbors = [x[0] for x in distanceVector[:15]]
        nearestNeighborLabels = []
        for num in nearestNeighbors:
            label = trainLabels[num]
            nearestNeighborLabels.append(label)
        prediction = stats.mode(nearestNeighborLabels)[0][0]
        predictionVector.append(prediction)
    
    # Compile the results
    resultDf = pd.DataFrame(index = testingSet.index, columns = [outputLabel, 'Prediction'])
    originalTestingDf = df.iloc[testingSet.index,:]
    resultDf['Primary Cell Class'] = originalTestingDf['Primary Cell Class']
    resultDf['Prediction'] = predictionVector
    
    count = 0
    for r in list(range(len(resultDf.index))):
        if resultDf.iloc[r,0] == resultDf.iloc[r,1]:
            count +=1
    
    # Calculate accuracy
    accuracy = count / len(resultDf.index)
    return accuracy

In [ ]:
# Test classification algorithm for species name, primary cell class, and primary brain region
predictors = ['Soma Surface','Number of Stems', 'Number of Bifurcations', 'Number of Branches', 'Overall Width', 'Overall Height', 'Overall Depth', 'Average Diameter', 'Total Length', 'Total Surface', 'Total Volume', 'Max Euclidean Distance', 'Max Path Distance', 'Max Branch Order', 'Average Contraction', 'Total Fragmentation', 'Partition Asymmetry', 'Average Rall\'s Ratio', 'Average Bifurcation Angle Local', 'Average Bifurcation Angle Remote', 'Fractal Dimension']

speciesAccuracy = neuronClassifier(predictors, 'Species Name')
cellClassAccuracy = neuronClassifier(predictors, 'Primary Cell Class')
brainRegionAccuracy = neuronClassifier(predictors, 'Primary Brain Region')


In [ ]:
# Classification Results
print('Species were classified with an accuracy of ' + str(speciesAccuracy))
print('Primary Cell Classes were classified with an accuracy of ' + str(cellClassAccuracy))
print('Primary Brain Regions were classified with an accuracy of ' + str(brainRegionAccuracy))